In [2]:
### count plot of all the class labels. 
import os
import seaborn as sns
import tensorflow
import matplotlib.pyplot as plt
import nltk
from nltk.chunk import ne_chunk
from tqdm import tqdm
import codecs
import re
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
%load_ext tensorboard
import tensorflow_addons as tfa
import pydot
import graphviz

In [3]:
with open('processed_data.pickle','rb') as fe_data_file:
    final_data=pickle.load(fe_data_file)

raw_text=final_data[0]
email_data=final_data[1]
subject_data=final_data[2]
processed_text=final_data[3]
file_name=final_data[4]
class_details=final_data[5]

data=pd.DataFrame(list(zip(raw_text,email_data,subject_data,processed_text,file_name,class_details)),
             columns=['text','email_id','subject','processed_text','file_name','class'])
x=data['email_id']+data['subject']+ data['processed_text']
y=data['class']

total_class=list(data['class'].unique())
class_labels=dict()
for i in range(len(total_class)):
    class_labels[i]=total_class[i]
    y[y==total_class[i]]=i

y = np.asarray(y).astype(np.float32)

In [4]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25, stratify=y)
x_train.shape

#https://towardsdatascience.com/nlp-preparing-text-for-deep-learning-model-using-tensorflow2-461428138657

tokenizer=tf.keras.preprocessing.text.Tokenizer(num_words=5000,filters='!"#$%&()*+,-./:;<=>?@[\\]^`{|}~\t\n',oov_token='UNK')
tokenizer.fit_on_texts(x_train)

x_train_tokenized=tokenizer.texts_to_sequences(x_train)
x_test_tokenized=tokenizer.texts_to_sequences(x_test)



max_values_in_list=[]
for j in x_train_tokenized:
    value=max(j)
    max_values_in_list.append(value)
    
input_dim=max(max_values_in_list)
input_dim

padded_x_train=tf.keras.preprocessing.sequence.pad_sequences(x_train_tokenized,maxlen=15000, padding='pre')
padded_x_test=tf.keras.preprocessing.sequence.pad_sequences(x_test_tokenized,maxlen=15000, padding='pre')

print(padded_x_train.shape)
print(padded_x_test.shape)

(14121, 15000)
(4707, 15000)


In [7]:
models=os.listdir('model_dash')
for m in models:
    model_hash=tf.keras.models.load_model('model_dash/'+m)
    print(m)
    print('Model_Performance')
    print(model_hash.evaluate(padded_x_test,y_test))
    print('*'*100)

weights_model_dash_epoch_0.hdf5
Model_Performance
148/148 [==============================] - 221s 1s/step - loss: 4.5029 - accuracy: 0.0814 - f1_score: 0.0042
[4.5028815269470215, 0.08136817812919617, 0.004229126498103142]
****************************************************************************************************
weights_model_dash_epoch_1.hdf5
Model_Performance
148/148 [==============================] - 245s 2s/step - loss: 6.1231 - accuracy: 0.0620 - f1_score: 0.0246
[6.123074054718018, 0.06203526630997658, 0.02463432587683201]
****************************************************************************************************
weights_model_dash_epoch_10.hdf5
Model_Performance
148/148 [==============================] - 254s 2s/step - loss: 19.8538 - accuracy: 0.0654 - f1_score: 0.0801
[19.853775024414062, 0.06543445587158203, 0.0801483541727066]
****************************************************************************************************
weights_model_dash_epoch_11.hd

KeyboardInterrupt: 